In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from unity import copy_file_to_unity_resources, format_resource_path
from utils import print_pretty
from hashing import hash_file, hash_string, hash_dict, hash_list
from package_graincloud import create_graincloud_metadata
from resources import create_package_resource, create_web_resource
from web_storage import upload_file_to_pinata, pinata_web_provider, aws_web_provider
import json

### Creating resources for a GrainCloud

In order to publish a GrainCloud, you must include links to the files/resources along with the metadata. These additionaly files include

- audio samples
- image thumbnails
- user session state
- sequences
- more to come...

This repo includes tools to upload samples to a web provider, such as IPFS (using Pinata), and automatically generate correct metadata.

In [3]:
local_file="E:/AudioClips/GranularTextures_Mini_Noiiz/118_ChainDrums_831.wav"
title = "Granular Textures - 118_ChainDrums_831 by Noiiz"
description="Granular Textures - 118_ChainDrums_831 by Noiiz"
title_hash = hash_string(title)

In [4]:
resource_web = create_web_resource(
    local_file=local_file,
    category="sample",
    web_provider=aws_web_provider, # or use a helper function
)

print_pretty(resource_web)

Autodetected MIME type: audio/wav
{
    "type": "audio/wav",
    "category": "sample",
    "location": "web",
    "uri": "https://timbrespace.s3.amazonaws.com/118_ChainDrums_831.wav",
    "hash": "f2448d416cd0804a",
    "bytes": 2190456
}


In [5]:
# web_provider must be a function that takes in a local file uri,
# and provides back a web uri where the resource has been uploaded to
resource_web = create_web_resource(
    local_file=local_file, 
    category="sample",
    web_provider=lambda x: upload_file_to_pinata(x, filename=title)["url"], # either use a lambda
    # web_provider=pinata_web_provider, # or use a helper function
)

print_pretty(resource_web)

{
    "type": "audio/wav",
    "category": "sample",
    "location": "web",
    "uri": "https://ipfs.io/ipfs/bafybeifzfxyjrs5wufnw6a6wbilj5up66lrk55fessoz7443kawtslloui",
    "hash": "9b2df6ee20edaaec5a0ed767fe51a0be0a20e7a9ce8fe723cd0487eabdb5f6fe",
    "bytes": 73859138
}


In [15]:
resource_package = create_package_resource(
    local_file=local_file, 
    category="sample",
    title=title,
)

print_pretty(resource_package)

{
    "type": "audio/wav",
    "category": "sample",
    "location": "package",
    "uri": "SamplePacks/Stuck Inside Your Side/StuckInsideYourSide-2-VOX",
    "hash": "9b2df6ee20edaaec5a0ed767fe51a0be0a20e7a9ce8fe723cd0487eabdb5f6fe",
    "bytes": 73859138
}


When loading a resource, TimbreSpaceVR will consider each resource with the same hash equivalent, and look for the easiest one to load. Depending on the platform, this will either be stored in appdata (android, windows), or playerprefs (webgl).

Description of the parameters:
- "type" : MIME type of the content, determines how the application should download/convert the file
- "category" : How TimbreSpaceVR will interpret the content. "sample" will be loaded as the audio for a GrainCloud
- "location" : The location where the resource is located. TSVR will load the resource easiest to load. Depending on the platform, this will either be stored in appdata (android, windows), or playerprefs (webgl).
- "uri" : Location of the actual file
- "hash" : sha256 hash of the file's contents to determine which files are unique
- "bytes" : Size of the file in bytes

### Building Metadata package for a GrainCloud

Now that resources have been generated, the final metadata package can be built. 

In [9]:
metadata = create_graincloud_metadata(
    local_file=local_file,
    title=title,
    description=description,
    parameters= { # these are the default parameters (definitions.DEFAULT_PARAMETERS)
        "xFeature": "MFCC_0",
        "yFeature": "MFCC_1",
        "zFeature": "MFCC_2",
        "rFeature": "MFCC_3",
        "gFeature": "MFCC_4",
        "bFeature": "MFCC_5",
        "scaleFeature": "RMS",
        "windowSize": 8192,
        "hopSize": 1024,
        "scaleMult" : 0.05,
        "scaleExp" : 0.1,
        "useHSV" : False,
        "posAxisScale" : [1,1,1]
    },
    # resources=[resource_web, resource_package],
    resources=[resource_web],
    creator={
        "name" : "Carl Moore",
        "website" : "https://carlmoore.xyz"
    },
    output_file=f"data/metadata/{hash_string(title)}.json"
)
print_pretty(metadata)

{
    "title": "Granular Textures - 118_ChainDrums_831 by Noiiz",
    "description": "Granular Textures - 118_ChainDrums_831 by Noiiz",
    "hash": "cbf199cddc91d4c6",
    "creator": {
        "name": "Carl Moore",
        "website": "https://carlmoore.xyz"
    },
    "parameters": {
        "xFeature": "MFCC_0",
        "yFeature": "MFCC_1",
        "zFeature": "MFCC_2",
        "rFeature": "MFCC_3",
        "gFeature": "MFCC_4",
        "bFeature": "MFCC_5",
        "scaleFeature": "RMS",
        "windowSize": 8192,
        "hopSize": 1024,
        "scaleMult": 0.05,
        "scaleExp": 0.1,
        "useHSV": false,
        "posAxisScale": [
            1,
            1,
            1
        ]
    },
    "resources": [
        {
            "type": "audio/wav",
            "category": "sample",
            "location": "web",
            "uri": "https://timbrespace.s3.amazonaws.com/118_ChainDrums_831.wav",
            "hash": "f2448d416cd0804a",
            "bytes": 2190456
        }

If you're ready to upload the metadata, go ahead!

In [10]:
from web_storage import upload_json_to_aws
from utils import print_pretty, save_json, load_json

hosted_metadata = "data/hosted-metadata.json"

res = upload_json_to_aws(metadata)

if res is not None:
    print_pretty(res)
    # add url and title to data/saved-metadata.json, and save the file
    saves = load_json(hosted_metadata)
    saves.append({
        "title" : title,
        "url" : res["url"]
    })
    save_json(saves, hosted_metadata)


{
    "object_key": "tmp56xoj0rc.json",
    "url": "https://timbrespace.s3.amazonaws.com/tmp56xoj0rc.json"
}


In [ ]:
aws_web_provider(json.dumps(metadata))

In [11]:
from web_storage import upload_json_to_pinata

upload_json_to_pinata(metadata, pinata_metadata={"name": f"{hash_string(title)}-metadata"})

{"pinataOptions": {"cidVersion": 1}, "pinataContent": "{\"title\": \"Stuck Inside Your Side\", \"description\": \"A song I wrote\", \"hash\": \"f4a29d8f3a179f76ffaa6182730dd5533dab44db7d2a37751d8001993a9e38e9\", \"creator\": {\"name\": \"Carl Moore\", \"website\": \"https://carlmoore.xyz\"}, \"parameters\": {\"xFeature\": \"MFCC_0\", \"yFeature\": \"MFCC_1\", \"zFeature\": \"MFCC_2\", \"rFeature\": \"MFCC_3\", \"gFeature\": \"MFCC_4\", \"bFeature\": \"MFCC_5\", \"scaleFeature\": \"RMS\", \"windowSize\": 8192, \"hopSize\": 8192, \"scaleMult\": 0.01, \"scaleExp\": 0.1, \"useHSV\": false, \"posAxisScale\": [1, 1, 1]}, \"resources\": [{\"type\": \"audio/wav\", \"category\": \"sample\", \"location\": \"web\", \"uri\": \"https://ipfs.io/ipfs/bafybeifzfxyjrs5wufnw6a6wbilj5up66lrk55fessoz7443kawtslloui\", \"hash\": \"9b2df6ee20edaaec5a0ed767fe51a0be0a20e7a9ce8fe723cd0487eabdb5f6fe\", \"bytes\": 73859138}, {\"type\": \"audio/wav\", \"category\": \"sample\", \"location\": \"package\", \"uri\": \

{'IpfsHash': 'bafkreicnaxcvhh6nvtyrmvojivtvlflea7hbm72rk7yipycivmxjt76bgq',
 'PinSize': 1148,
 'Timestamp': '2023-04-02T05:59:41.251Z',
 'url': 'https://ipfs.io/ipfs/bafkreicnaxcvhh6nvtyrmvojivtvlflea7hbm72rk7yipycivmxjt76bgq'}

Format a file for the Unity resources package (needed for developer only)

In [ ]:
uri = copy_file_to_unity_resources("C:/Users/Carl/Desktop/carl_sc.png", "test")
test = format_resource_path(uri)
print(test)